In [1]:
#importing all the required libraries

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image



In [2]:
# Load the pre-trained VGG16 model with pre-trained weights
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained VGG16 model
for layer in vgg.layers:
    layer.trainable = False

# Add new layers at the end of the pre-trained VGG16 model
x = Flatten()(vgg.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(4, activation='softmax')(x)

# Create the new model
model = Model(inputs=vgg.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

58889256/58889256 [==============================] - 0s 0us/step


In [3]:
#create dataset for new model training

data_dir = "/content/drive/MyDrive/VR Gallery Art"
batch_size = 32
img_size = (224, 224)

datagen = ImageDataGenerator(rescale=1./255, vertical_flip = True, horizontal_flip= True, validation_split=0.2)
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 244 images belonging to 4 classes.
Found 59 images belonging to 4 classes.


In [4]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10
)

Epoch 1/10
7/7 [==============================] - 114s 15s/step - loss: 3.8524 - accuracy: 0.3302 - val_loss: 0.7171 - val_accuracy: 0.6562
Epoch 2/10
7/7 [==============================] - 101s 15s/step - loss: 2.5143 - accuracy: 0.5613 - val_loss: 0.5244 - val_accuracy: 0.7812
Epoch 3/10
7/7 [==============================] - 99s 14s/step - loss: 1.7408 - accuracy: 0.6509 - val_loss: 0.3502 - val_accuracy: 0.8438
Epoch 4/10
7/7 [==============================] - 106s 16s/step - loss: 1.0737 - accuracy: 0.7075 - val_loss: 0.2532 - val_accuracy: 0.9375
Epoch 5/10
7/7 [==============================] - 106s 16s/step - loss: 0.8070 - accuracy: 0.7925 - val_loss: 0.6102 - val_accuracy: 0.7812
Epoch 6/10
7/7 [==============================] - 103s 15s/step - loss: 0.5106 - accuracy: 0.8393 - val_loss: 0.2950 - val_accuracy: 0.8438
Epoch 7/10
7/7 [==============================] - 99s 14s/step - loss: 0.5594 - accuracy: 0.8302 - val_loss: 0.1772 - val_accuracy: 0.9688
Epoch 8/10
7/7 [======

In [5]:
# evaluate the model on the test set
loss, accuracy = model.evaluate(validation_generator, steps=len(validation_generator))

# print the metrics
print("Test loss:", loss)
print("Test accuracy:", accuracy)

2/2 [==============================] - 24s 11s/step - loss: 0.1875 - accuracy: 0.9153
Test loss: 0.18753384053707123
Test accuracy: 0.9152542352676392


In [ ]:
# Define the file path for saving the model
model_path = '/content/drive/MyDrive/MyModelVGG.h5'

model.save(model_path)